In [43]:
%pylab inline
import itertools as it
import nltk
tokenizer = nltk.tokenize.word_tokenize
import collections
import operator
import gzip

Populating the interactive namespace from numpy and matplotlib


/home/nuno-mota/Programas/Anaconda/Installation/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [44]:
def column(matrix, i):
    return [row[i] for row in matrix]

def get_types_list(formatted_language_corpus):
    language_types_list = []
    for sentence in formatted_language_corpus:
        for word in sentence:
            if (word not in language_types_list):
                language_types_list.append(word)
                
    return language_types_list

def get_types_list(formatted_language_corpus):
    language_types_list = []
    for sentence in formatted_language_corpus:
        for word in sentence:
            if (word not in language_types_list):
                language_types_list.append(word)
                
    return language_types_list

In [45]:
def escape(str):
    str = str.replace("&lt;", "")
    str = str.replace("lt;", "")
    str = str.replace("&gt;", "")
    str = str.replace("gt;", "")
    str = str.replace("&amp;", "")
    str = str.replace("amp;", "")
    str = str.replace("&quot;", "")
    str = str.replace("quot;", "")
    return str

In [46]:
eng_ind = collections.defaultdict(list)
ger_ind = collections.defaultdict(list)
fre_ind = collections.defaultdict(list)

start = '<seg id="'
end = '" type'
whitespace = "\r\n\t"

high_sentence1 = "English.xml"
high_sentence2 = "German.xml"
high_sentence3 = "French.xml"
low_sentence   = "Spanish.xml"

# Read the segments of the English, German, and French bible
with open(high_sentence1) as f:
    line = f.readlines()
    for i in range(len(line)):
        if "seg id=" in line[i] and "seg id=" not in line[i+1]:
            res = escape(line[i+1])
            eng_ind[line[i][line[i].find(start)+len(start):line[i].find(end)]].append(res.strip(whitespace))
            
with open(high_sentence2) as f:
    line = f.readlines()
    for i in range(len(line)):
        if "seg id=" in line[i] and "seg id=" not in line[i+1]: 
            res = escape(line[i+1])
            ger_ind[line[i][line[i].find(start)+len(start):line[i].find(end)]].append(res.strip(whitespace))

with open(high_sentence3) as f:
    line = f.readlines()
    for i in range(len(line)):
        if "seg id=" in line[i] and "seg id=" not in line[i+1]:
            res = escape(line[i+1])
            fre_ind[line[i][line[i].find(start)+len(start):line[i].find(end)]].append(res.strip(whitespace))            

# read the segments of low resource sentence
with open(low_sentence) as f:
    line = f.readlines()
    for i in range(len(line)):
        if "seg id=" in line[i] and "seg id=" not in line[i+1]:
            res = escape(line[i+1])
            eng_ind[line[i][line[i].find(start)+len(start):line[i].find(end)]].append(res.strip(whitespace))
            ger_ind[line[i][line[i].find(start)+len(start):line[i].find(end)]].append(res.strip(whitespace))
            fre_ind[line[i][line[i].find(start)+len(start):line[i].find(end)]].append(res.strip(whitespace))

# remove sentence that does not have a translation
keys_to_remove = [key for key, value in eng_ind.iteritems() if len(eng_ind[key]) == 1]
keys_to_remove2 = [key for key, value in ger_ind.iteritems() if len(ger_ind[key]) == 1]
keys_to_remove3 = [key for key, value in fre_ind.iteritems() if len(fre_ind[key]) == 1]

for key in keys_to_remove:
    del eng_ind[key]
for key in keys_to_remove2:
    del ger_ind[key]
for key in keys_to_remove3:
    del fre_ind[key]

In [47]:
training_set = None
training_set_2 = None
training_set_3 = None
test_set = None
test_set_2 = None
test_set_3 = None
keys_test_set = None


def pick_test_sentences(number_of_test_sentences):
    
    global test_set
    global test_set_2
    global test_set_3
    global keys_test_set
    
    test_set = []
    test_set_2 = []
    test_set_3 = []
    
    keys_test_set = []
    while(len(keys_test_set) != number_of_test_sentences):
        temp = random.choice(eng_ind.keys(), number_of_test_sentences - len(keys_test_set))
        for key in temp:
            if(key in eng_ind.keys() and key in ger_ind.keys() and key in fre_ind.keys() and
               key not in keys_test_set):
                keys_test_set.append(key)
                
    for key in keys_test_set:
        test_set.append(eng_ind[key])
        test_set_2.append(ger_ind[key])
        test_set_3.append(fre_ind[key])
        
    return None


def pick_training_sentences(number_of_training_sentences):
    
    global training_set
    global training_set_2
    global training_set_3
    global keys_test_set
    
    
    training_set = []
    training_set_2 = [] 
    training_set_3 = []


    keys_training_set = []
    while(len(keys_training_set) != number_of_training_sentences):
        temp = random.choice(eng_ind.keys(), number_of_training_sentences - len(keys_training_set))
        for key in temp:
            if(key in eng_ind.keys() and key in ger_ind.keys() and key in fre_ind.keys() and
               key not in keys_test_set and key not in keys_training_set):
                keys_training_set.append(key)

        
    for key in keys_training_set:
        training_set.append(eng_ind[key])
        training_set_2.append(ger_ind[key])
        training_set_3.append(fre_ind[key])
            
    
    return None
            
#pick_training_sentences(100)

In [48]:
import pickle
def save_training_set(number):
    global training_set
    global training_set_2
    global training_set_3
    with open("experiment_data/english_" + str(number) + ".txt", 'wb') as f:
        pickle.dump(training_set, f)
    with open("experiment_data/german_" + str(number) + ".txt", 'wb') as f:
        pickle.dump(training_set_2, f)
    with open("experiment_data/french_" + str(number) + ".txt", 'wb') as f:
        pickle.dump(training_set_3, f)

In [49]:
import pickle
def save_test_set():
    global test_set
    global test_set_2
    global test_set_3
    with open("experiment_data/english_test.txt", 'wb') as f:
        pickle.dump(test_set, f)
    with open("experiment_data/german_test.txt", 'wb') as f:
        pickle.dump(test_set_2, f)
    with open("experiment_data/french_test.txt", 'wb') as f:
        pickle.dump(test_set_3, f)

In [50]:
def dump_model(pt_filename, pt):
    with open("experiment_data/model/" + pt_filename, 'wb') as handle:
        pickle.dump(pt, handle)

In [51]:
english_token_train = None
spanish_token_train = None
german_token_train = None
spanish_token_train_2 = None
french_token_train = None
spanish_token_train_3 = None
english_token_test = None
spanish_token_test = None
german_token_test = None
spanish_token_test_2 = None
french_token_test = None
spanish_token_test_3 = None

language_train_pairs = None
language_test_pairs = None
language_train_pairs_2 = None
language_test_pairs_2 = None
language_train_pairs_3 = None
language_test_pairs_3 = None


def tokenize_test():

    global english_token_test
    global german_token_test
    global french_token_test
    global spanish_token_test
    global language_test_pairs
    global language_test_pairs_2
    global language_test_pairs_3
    
    english_token_test = []
    german_token_test = []
    french_token_test = []
    spanish_token_test = []
    spanish_token_test_2 = []
    spanish_token_test_3 = []
    

    for test in test_set:
        temp = "null " + test[0]
        temp2 = test[1] + "."
        english_token_test.append(tokenizer(temp.lower().decode('utf-8')))
        spanish_token_test.append(tokenizer(temp2.lower().decode('utf-8')))


    for test in test_set_2:
        temp = "null " + test[0]
        temp2 = test[1] + "."
        german_token_test.append(tokenizer(temp.lower().decode('utf-8')))
        spanish_token_test_2.append(tokenizer(temp2.lower().decode('utf-8')))


    for test in test_set_3:
        temp = "null " + test[0]
        temp2 = test[1] + "."
        french_token_test.append(tokenizer(temp.lower().decode('utf-8')))
        spanish_token_test_3.append(tokenizer(temp2.lower().decode('utf-8')))
        
        
    language_test_pairs = np.column_stack((np.array(english_token_test), np.array(spanish_token_test)))
    language_test_pairs_2 = np.column_stack((np.array(german_token_test), np.array(spanish_token_test_2)))
    language_test_pairs_3 = np.column_stack((np.array(french_token_test), np.array(spanish_token_test_3)))
    
    
    return None
    
    
    
def tokenize():
    
    global english_token_train
    global spanish_token_train
    global german_token_train
    global spanish_token_train_2
    global french_token_train
    global spanish_token_train_3
    global spanish_token_test_2
    global spanish_token_test_3
    global language_train_pairs
    global language_train_pairs_2
    global language_train_pairs_3
    
    english_token_train = []
    spanish_token_train = []
    german_token_train = []
    spanish_token_train_2 = []
    french_token_train = []
    spanish_token_train_3 = []
    
    for train in training_set:
        temp = "null " + train[0]
        temp2 = train[1] + "."
        english_token_train.append(tokenizer(temp.lower().decode('utf-8')))
        spanish_token_train.append(tokenizer(temp2.lower().decode('utf-8')))


    for train in training_set_2:
        temp = "null " + train[0]
        temp2 = train[1] + "."
        german_token_train.append(tokenizer(temp.lower().decode('utf-8')))
        spanish_token_train_2.append(tokenizer(temp2.lower().decode('utf-8')))


    for train in training_set_3:
        temp = "null " + train[0]
        temp2 = train[1] + "."
        french_token_train.append(tokenizer(temp.lower().decode('utf-8')))
        spanish_token_train_3.append(tokenizer(temp2.lower().decode('utf-8')))


    language_train_pairs = np.column_stack((np.array(english_token_train), np.array(spanish_token_train)))
    language_train_pairs_2 = np.column_stack((np.array(german_token_train), np.array(spanish_token_train_2)))
    language_train_pairs_3 = np.column_stack((np.array(french_token_train), np.array(spanish_token_train_3)))
    
    
    return None

In [53]:
english_types = None
german_types  = None
french_types  = None
spanish_types = None

def get_types():
    global english_types
    global german_types
    global french_types
    global spanish_types
    
    english_types = get_types_list(english_token_train)
    german_types  = get_types_list(german_token_train)
    french_types  = get_types_list(french_token_train)
    spanish_types = get_types_list(spanish_token_train)

    print "\nNumber of English types = ", len(english_types)
    print "Number of German types  = ", len(german_types)
    print "Number of French types  = ", len(french_types)
    print "Number of Spanish types = ", len(spanish_types)
    
    return None

In [54]:
def EM_translation_probabilities(formatted_language_pairs_corpus):
    origin_language_types = get_types_list(column(formatted_language_pairs_corpus, 0))
    target_language_types = get_types_list(column(formatted_language_pairs_corpus, 1))
    
    parameters_t = {}
    shape_count_parameters_t = {}
    
    for origin_type in origin_language_types:
        parameters_t[origin_type] = {}
        shape_count_parameters_t[origin_type] = {}
        
    for entry in formatted_language_pairs_corpus:
        for origin_word in entry[0]:
            for target_word in entry[1]:
                parameters_t[origin_word][target_word] = pow(10, -16)#np.random.uniform(pow(10, -16), pow(10, -10))
                shape_count_parameters_t[origin_word][target_word] = 0.0
    
    i = 20
    
    while (i > 0.001):#JUST 3 DECIMAL PLACES SEEMS TO BE FINE
        
        #Expectation Step
        counts = {}
        for origin_word in shape_count_parameters_t:
            counts[origin_word] = {}
            for target_word in shape_count_parameters_t[origin_word]:
                counts[origin_word][target_word] = 0
                
        total_origin = {}
        for origin_type in origin_language_types:
            total_origin[origin_type] = 0.0
        
        for entry in formatted_language_pairs_corpus:
            target_word_total = {}
            for target_word in entry[1]:
                target_word_total[target_word] = 0.0
                for origin_word in entry[0]:
                    target_word_total[target_word] += parameters_t[origin_word][target_word]
        
            for target_word in entry[1]:
                for origin_word in entry[0]:
                    counts[origin_word][target_word] += parameters_t[origin_word][target_word]/target_word_total[target_word]
                    total_origin[origin_word] += parameters_t[origin_word][target_word]/target_word_total[target_word]
        
        
        
        #Maximization Step
        i = 0
        parameters_t_new = {}    
        for origin_word in shape_count_parameters_t:
            parameters_t_new[origin_word] = {}
            for target_word in shape_count_parameters_t[origin_word]:
                parameters_t_new[origin_word][target_word] = counts[origin_word][target_word]/total_origin[origin_word]
                temp = abs(parameters_t_new[origin_word][target_word] - parameters_t[origin_word][target_word])
                if (i < temp):
                    i = temp

                    
        parameters_t = parameters_t_new
                    
                   
    ordered_parameters_t = {}
    
    for origin_type in origin_language_types:
        parameters_t_for_index = []
        word_for_index = []
        word_for_value = []
        i = 0
        for key in shape_count_parameters_t[origin_type]:
            word_for_index.append(key)
            parameters_t_for_index.append([i, parameters_t_new[origin_type][key]])
            i += 1
        parameters_t_for_index = np.array(parameters_t_for_index)
        ordered_parameters_t[origin_type] = parameters_t_for_index[parameters_t_for_index[:,1].argsort()[::-1]] #Nuno'S stolen magic
        
        
        for i in range(len(ordered_parameters_t[origin_type])):
            word_for_value.append([word_for_index[int(ordered_parameters_t[origin_type][i][0])], ordered_parameters_t[origin_type][i][1]])
        ordered_parameters_t[origin_type] = word_for_value
    
    return ordered_parameters_t, parameters_t

In [55]:
#for element in ordered_parameters_t_IBM1_eng["to"]:
#    print element

In [56]:
def calculate_delta(parameters_t, parameters_q, source_sentence, target_sentence):
    length_source = len(source_sentence)
    length_target = len(target_sentence)
    
    delta_matrix = np.zeros((length_source, length_target))
    normalizer = np.zeros(length_source)
    
    for i, source_word in enumerate(source_sentence):
        for j, target_word in enumerate(target_sentence):
            t = parameters_t[source_word][target_word]
            q = parameters_q[length_source][length_target][i][j]
            t_q = t*q
            normalizer[i] += t_q
            delta_matrix[i][j] = t_q
            
    return delta_matrix / normalizer[:, None]

def EM_translation_probabilities_IBM2(formatted_language_pairs_corpus):
    origin_language_types = get_types_list(column(formatted_language_pairs_corpus, 0))
    target_language_types = get_types_list(column(formatted_language_pairs_corpus, 1))
    
    
    parameters_t = {}
    parameters_q = {}
    
    for origin_type in origin_language_types:
        parameters_t[origin_type] = {}
  
    for entry in formatted_language_pairs_corpus:
        for origin_word in entry[0]:
            for target_word in entry[1]:
                parameters_t[origin_word][target_word] = pow(10, -16)
        
        if len(entry[0]) not in parameters_q:
            parameters_q[len(entry[0])] = {}
        if len(entry[1]) not in parameters_q[len(entry[0])]:
            parameters_q[len(entry[0])][len(entry[1])] = np.full((len(entry[0]), len(entry[1])), pow(10, -16))

    x = 20
    z = 20
    
    while (x > 0.01 or z > 0.01):#JUST 2 DECIMAL PLACES SEEMS TO BE FINE, otherwise takes too long
        
        #Set counts to 0
        counts = {}
        total_origin = {}
        for origin_word in parameters_t:
            counts[origin_word] = {}
            total_origin[origin_word] = 0.0
            for target_word in parameters_t[origin_word]:
                counts[origin_word][target_word] = 0.0
            
            
        q_counts = {}
        q_counts_target = {}
        for key_origin in parameters_q:
            q_counts[key_origin] = {}
            q_counts_target[key_origin] = {}
            for key_target in parameters_q[key_origin]:
                q_counts[key_origin][key_target] = np.zeros((key_origin, key_target))
                q_counts_target[key_origin][key_target] = np.zeros(key_target)
        
        
        #Expectation Step
        for entry in formatted_language_pairs_corpus:
            delta = calculate_delta(parameters_t, parameters_q, entry[0], entry[1])
            source_length = len(entry[0])
            target_length = len(entry[1])
            for i, origin_word in enumerate(entry[0]):
                for j, target_word in enumerate(entry[1]):
                    val = delta[i][j]
                    counts[origin_word][target_word] += val
                    total_origin[origin_word] += val
                    q_counts[source_length][target_length][i][j] += val
                    q_counts_target[source_length][target_length][j] += val
        
        
        #Maximization Step
        x = 0
        parameters_t_new = {}    
        for origin_word in parameters_t:
            parameters_t_new[origin_word] = {}
            c_o = total_origin[origin_word]
            for target_word in parameters_t[origin_word]:
                c   = counts[origin_word][target_word]
                parameters_t_new[origin_word][target_word] = c/c_o
                temp = abs(parameters_t_new[origin_word][target_word] - parameters_t[origin_word][target_word])
                if (x < temp):
                    x = temp
                    
        parameters_t = parameters_t_new
                
            
        parameters_q_new = {}
        z = 0
        for key_origin in parameters_q:
            parameters_q_new[key_origin] = {}
            for key_target in parameters_q[key_origin]:
                mat = q_counts[key_origin][key_target] / q_counts_target[key_origin][key_target][None, :]
                parameters_q_new[key_origin][key_target] = mat
                temp = abs((parameters_q_new[key_origin][key_target] - parameters_q[key_origin][key_target]).max())
                if (z < temp):
                    z = temp
    
        parameters_q = parameters_q_new
                   
    ordered_parameters_t = {}
    
    for origin_type in origin_language_types:
        parameters_t_for_index = []
        word_for_index = []
        word_for_value = []
        i = 0
        for key in parameters_t_new[origin_type]:
            word_for_index.append(key)
            parameters_t_for_index.append([i, parameters_t_new[origin_type][key]])
            i += 1
        parameters_t_for_index = np.array(parameters_t_for_index)
        ordered_parameters_t[origin_type] = parameters_t_for_index[parameters_t_for_index[:,1].argsort()[::-1]] #Nuno'S stolen magic
        
        
        for i in range(len(ordered_parameters_t[origin_type])):
            word_for_value.append([word_for_index[int(ordered_parameters_t[origin_type][i][0])], ordered_parameters_t[origin_type][i][1]])
        ordered_parameters_t[origin_type] = word_for_value
    
    return ordered_parameters_t, parameters_t, parameters_q

In [57]:
#for element in ordered_parameters_t_IBM2_eng["from"]:
#    print element

In [58]:
def IBM1(origin_sentence, target_sentence, parameters_t):
    
    best_alignment = [0]*len(target_sentence)
    for i,target_word in enumerate(target_sentence):
        max = 0
        for j, source_word in enumerate(origin_sentence):
            if source_word in parameters_t and target_word in parameters_t[source_word]:
                
                if max < parameters_t[source_word][target_word]:
                    best_alignment[i] = j
                    max = parameters_t[source_word][target_word]
                    
    return best_alignment

'origin1 = language_test_pairs[0][0]\ntarget1 = language_test_pairs[0][1]\nresult1 = IBM1(origin1, target1, parameters_t_IBM1_eng)'

In [59]:
def IBM2(origin_sentence, target_sentence, parameters_t, parameters_q):
    
    _DEBUG = True
    
    best_alignment = [0]*len(target_sentence)
    for i, target_word in enumerate(target_sentence):
        max = 0
        
        if _DEBUG:
            print "NEW TARGET WORD PICKED: ", target_word
            
        for j, source_word in enumerate(origin_sentence):
            if source_word in parameters_t and target_word in parameters_t[source_word]:
                if(len(origin_sentence) not in parameters_q):
                    q = 1
                    if _DEBUG:
                        print "UNKNOWN SENTENCE LENGTH"
                else:
                    if(len(target_sentence) not in parameters_q[len(origin_sentence)]):
                        q = 1
                        if _DEBUG:
                            print "UNKNOWN SENTENCE LENGTH"
                    else:
                        q = parameters_q[len(origin_sentence)][len(target_sentence)][j][i]
                temp = q * parameters_t[source_word][target_word]
                
                if _DEBUG:
                        print temp, q, parameters_t[source_word][target_word]
                        
                if max < temp :
                    best_alignment[i] = j
                    max = temp
            else:
                t = 1
                if source_word not in parameters_t:
                    if _DEBUG:
                        print "UNKNOWN SOURCE WORD: ", source_word
                else:
                    if target_word not in parameters_t[source_word]:
                        if _DEBUG:
                            print "UNKNOWN TARGET WORD", target_word

                if(len(origin_sentence) not in parameters_q):
                    q = 0
                    if _DEBUG:
                        print "UNKNOWN SENTENCE LENGTH"
                else:
                    if(len(target_sentence) not in parameters_q[len(origin_sentence)]):
                        q = 0
                        if _DEBUG:
                            print "UNKNOWN SENTENCE LENGTH"
                    else:
                        q = parameters_q[len(origin_sentence)][len(target_sentence)][j][i]
                        
                temp = q * t
            
                if _DEBUG:
                    print temp, q, t
                    
                if max < temp:
                    best_alignment[i] = j
                    max = temp
                    
    return best_alignment

'origin2 = language_test_pairs[0][0]\ntarget2 = language_test_pairs[0][1]\nresult2 = IBM2(origin2, target2, parameters_t_IBM2_eng, parameters_q_IBM2_eng)'

In [ ]:
'''origin1 = language_test_pairs[0][0]
#print origin1
target1 = language_test_pairs[0][1]
#print target1
result1 = IBM1(origin1, target1, parameters_t_IBM1_eng)

print "----------"

result_print = []
for i in range(len(result1)):
    result_print.append(str(target1[i].encode('utf-8') + " " + str(result1[i])))
print result_print 
result_origin = []
for i in range(len(origin1)):
    result_origin.append(str(origin1[i].encode('utf-8') + " " + str(i)))
print result_origin'''

In [62]:
'''origin2 = language_test_pairs[0][0]
#print origin2
target2 = language_test_pairs[0][1]
#print target2
result2 = IBM2(origin2, target2, parameters_t_IBM2_eng, parameters_q_IBM2_eng)

print "----------"

result_print = []
for i in range(len(result2)):
    result_print.append(str(target2[i].encode('utf-8') + " " + str(result2[i])))
print result_print 
result_origin = []
for i in range(len(language_test_pairs[0][0])):
    result_origin.append(str(origin2[i].encode('utf-8') + " " + str(i)))
print result_origin'''

'result_print = []\nfor i in range(len(result2)):\n    result_print.append(str(target2[i].encode(\'utf-8\') + " " + str(result2[i])))\nprint result_print \nresult_origin = []\nfor i in range(len(language_test_pairs[0][0])):\n    result_origin.append(str(origin2[i].encode(\'utf-8\') + " " + str(i)))\nprint result_origin'

In [63]:
import os
from nltk.tag.stanford import StanfordPOSTagger
java_path = "C:\Program Files\Java\jdk1.8.0_111"
os.environ['JAVAHOME'] = java_path

In [64]:
def pos_tagger_english(sentence):
    path_to_model = "stanford-postagger-full-2015-12-09/models/english-bidirectional-distsim.tagger"
    path_to_jar = "stanford-postagger-full-2015-12-09/stanford-postagger.jar"
    tagger=StanfordPOSTagger(path_to_model, path_to_jar)
    tagger.java_options='-mx1000m'
    return tagger.tag(sentence)
    
def pos_tagger_german(sentence):
    path_to_model = "stanford-postagger-full-2015-12-09/models/german-hgc.tagger"
    path_to_jar = "stanford-postagger-full-2015-12-09/stanford-postagger.jar"
    tagger=StanfordPOSTagger(path_to_model, path_to_jar)
    tagger.java_options='-mx1000m'
    return tagger.tag(sentence)

def pos_tagger_french(sentence):
    path_to_model = "stanford-postagger-full-2015-12-09/models/french.tagger"
    path_to_jar = "stanford-postagger-full-2015-12-09/stanford-postagger.jar"
    tagger=StanfordPOSTagger(path_to_model, path_to_jar)
    tagger.java_options='-mx1000m'
    return tagger.tag(sentence)

def pos_tagger_spanish(sentence):
    path_to_model = "stanford-postagger-full-2015-12-09/models/spanish.tagger"
    path_to_jar = "stanford-postagger-full-2015-12-09/stanford-postagger.jar"
    tagger=StanfordPOSTagger(path_to_model, path_to_jar)
    tagger.java_options='-mx1000m'
    return tagger.tag(sentence)

In [65]:
eng_dict = {}
with open("postag-highsentence/english.map") as f:
    lines = f.readlines()
    for line in lines:
        res = line.split()
        eng_dict[res[0]] = res[1]

ger_dict = {}
with open("postag-highsentence/german.map") as f:
    lines = f.readlines()
    for line in lines:
        res = line.split()
        ger_dict[res[0]] = res[1]

fre_dict = {}
with open("postag-highsentence/french.map") as f:
    lines = f.readlines()
    for line in lines:
        res = line.split()
        if(res[0] == "PONCT"):
            fre_dict["PUNC"] = res[1]
        else:
            fre_dict[res[0]] = res[1]
            
spa_dict = {}
with open("postag-highsentence/spanish.map") as f:
    lines = f.readlines()
    for line in lines:
        res = line.split()
        spa_dict[res[0]] = res[1]

In [66]:
def mapper(sentences, universal_dict):
    for i, sentence in enumerate(sentences):
        for j, word_tag in enumerate(sentence):
            if(word_tag[1] in universal_dict):
                temp = list(sentences[i][j])
                temp[1] = universal_dict[word_tag[1]]
                sentences[i][j] = tuple(temp)
    return sentences

In [67]:
english_sentences_universal = None
german_sentences_universal  = None
french_sentences_universal  = None
spanish_sentences_universal = None

def tag_test_sentences():
    
    global english_sentences_universal
    global german_sentences_universal
    global french_sentences_universal
    global spanish_sentences_universal

    english_sentences = []
    for sentence in language_test_pairs:
        english_sentences.append(pos_tagger_english(sentence[0]))
    english_sentences_universal = mapper(english_sentences, eng_dict)
    
    # german will take some time to tag
    german_sentences = []
    for sentence in language_test_pairs_2:
        german_sentences.append(pos_tagger_german(sentence[0]))
    german_sentences_universal = mapper(german_sentences, ger_dict)
    
    french_sentences = []
    for sentence in language_test_pairs_3:
        french_sentences.append(pos_tagger_french(sentence[0]))
    french_sentences_universal = mapper(french_sentences, fre_dict)
    
    spanish_sentences = []
    for sentence in language_test_pairs:
        spanish_sentences.append(pos_tagger_spanish(sentence[1]))
    spanish_sentences_universal = mapper(spanish_sentences, spa_dict)

In [71]:
source_sentences = None
language_family = None

def get_source_sentences_tagged():
    
    global source_sentences
    global language_family
    
    source_sentences = []
    for i in range(len(english_sentences_universal)):
        source_sentences.append([[english_sentences_universal[i], german_sentences_universal[i],
                                  french_sentences_universal[i]], language_test_pairs[i][1]])

    english_language_families = ["Indo-European", "Germanic", "West"]
    german_language_families  = ["Indo-European", "Germanic", "West"]
    french_language_families  = ["Indo-European", "Italic", "Romance"]
    spanish_language_families = ["Indo-European", "Italic", "Romance"]
    language_family = [english_language_families, german_language_families, 
                       french_language_families, spanish_language_families]
#print source_sentences

In [72]:
def quorum(tags):
    data = collections.Counter(tags)
    return data.most_common(1)[0][0]

def propagateTags(source_sentences, target_sentence, parameters_t_tables, 
                  parameters_q_tables, weighted, language_family):
    alignments = []
    tagged_target_sentence = []
    for word in target_sentence:
        tagged_target_sentence.append([word])
    for i, sentences in enumerate(source_sentences):
        untagged_sentence = []
        for tagged_word in sentences:
            untagged_sentence.append(tagged_word[0])
        if parameters_q_tables is None:
            alignments.append(IBM1(untagged_sentence, target_sentence, parameters_t_tables[i]))
        else:
            alignments.append(IBM2(untagged_sentence, target_sentence, parameters_t_tables[i], parameters_q_tables[i]))

            
    if not weighted:    
        for i, word in enumerate(target_sentence):
            tags = []
            for j, alignment in enumerate(alignments):
                tag = source_sentences[j][alignment[i]][1]
                tags.append(tag)
            tagged_target_sentence[i].append(quorum(tags))
            for elem in alignments:
                tagged_target_sentence[i].append(elem[i])
                
    else:
        for i, word in enumerate(target_sentence):
            tags = {}
            for j, alignment in enumerate(alignments):
                tag = source_sentences[j][alignment[i]][1]
                
                if(language_family[0][j][0] == language_family[0][-1][0]):
                    if(language_family[0][j][1] == language_family[0][-1][1]):
                        if(language_family[0][j][2] == language_family[0][-1][2]):
                            val = 2.5
                        else:
                            val = 2
                    else:
                        val = 1.5
                else:
                    val = 1
                    
                if tag in tags:
                    tags[tag] += val
                else:
                    tags[tag] = val
            
            tags_l = []
            for tag in tags:
                tags[tag] = int(tags[tag])
                while(tags[tag] > 0):
                    tags_l.append(tag)
                    tags[tag] -= 1
            
            tagged_target_sentence[i].append(quorum(tags_l))
            for elem in alignments:
                tagged_target_sentence[i].append(elem[i])
    
    
    return tagged_target_sentence


def tag_low_resource_sentences(sentence_groups, parameters_t_tables, parameters_q_tables,
                               weighted, language_family, unique_language_index):
    tagged_target_sentences = []
    
    if unique_language_index is None:
        for entry in sentence_groups:
            tagged_target_sentences.append(propagateTags(entry[0], entry[1], parameters_t_tables,
                                                         parameters_q_tables, weighted, language_family))
    else:
        for entry in sentence_groups:
            tagged_target_sentences.append(propagateTags([entry[0][unique_language_index]], entry[1],
                                                         parameters_t_tables, parameters_q_tables, 
                                                         weighted, language_family))

    return tagged_target_sentences

In [75]:
def calculate_accuracy(source_sentences, spanish_sentences_universal):
    
    accuracies = []
    
    for l in range(len(source_sentences[0][0])):
        temp_result = tag_low_resource_sentences(source_sentences, parameters_t_all_IBM1,
                                             None, False, None, l)
        
        i = 0.0
        total_words = 0.0
        for j in range(len(temp_result)):
            for k in range(len(temp_result[j])):
                if temp_result[j][k][1] == spanish_sentences_universal[j][k][1]:
                    i += 1
            total_words += k

        language_l = i / total_words
        accuracies.append(language_l)
    
    
    result_IBM1 = tag_low_resource_sentences(source_sentences, parameters_t_all_IBM1,
                                             None, False, None, None)
    i = 0.0
    total_words = 0.0
    for j in range(len(result_IBM1)):
        for k in range(len(result_IBM1[j])):
            if result_IBM1[j][k][1] == spanish_sentences_universal[j][k][1]:
                i += 1
        total_words += k
    
    IBM1 = i / total_words
    accuracies.append(IBM1)
    #print "IBM1: ", IBM1
    
    
    
    result_IBM1 = tag_low_resource_sentences(source_sentences, parameters_t_all_IBM1,
                                             None, True, language_family, None)
    i = 0.0
    total_words = 0.0
    for j in range(len(result_IBM1)):
        for k in range(len(result_IBM1[j])):
            if result_IBM1[j][k][1] == spanish_sentences_universal[j][k][1]:
                i += 1
        total_words += k
    
    IBM1_WEIGHTED = i / total_words
    accuracies.append(IBM1_WEIGHTED)
    #print "IBM1 WEIGHTED: ", IBM1_WEIGHTED
    
    
    
    
    '''for l in range(len(source_sentences[0][0])):
        temp_result = tag_low_resource_sentences(source_sentences, parameters_t_all_IBM2,
                                             parameters_q_all_IBM2, False, None, l)
        
        i = 0.0
        total_words = 0.0
        for j in range(len(temp_result)):
            for k in range(len(temp_result[j])):
                if temp_result[j][k][1] == spanish_sentences_universal[j][k][1]:
                    i += 1
            total_words += k

        language_l = i / total_words
        accuracies.append(language_l)
    
    
    result_IBM2 = tag_low_resource_sentences(source_sentences, parameters_t_all_IBM2,
                                             parameters_q_all_IBM2, False, None, None)
    i = 0.0
    total_words = 0.0
    for j in range(len(result_IBM2)):
        for k in range(len(result_IBM2[j])):
            if result_IBM2[j][k][1] == spanish_sentences_universal[j][k][1]:
                i += 1
        total_words += k
        
    IBM2 = i / total_words
    accuracies.append(IBM2)
    #print "IBM2: ", IBM2
    
    
    result_IBM2 = tag_low_resource_sentences(source_sentences, parameters_t_all_IBM2,
                                             parameters_q_all_IBM2, True, language_family, None)
    i = 0.0
    total_words = 0.0
    for j in range(len(result_IBM2)):
        for k in range(len(result_IBM2[j])):
            if result_IBM2[j][k][1] == spanish_sentences_universal[j][k][1]:
                i += 1
        total_words += k
        
    IBM2_WEIGHTED = i / total_words
    accuracies.append(IBM2_WEIGHTED)
    #print "IBM2_WEIGHTED: ", IBM2_WEIGHTED'''
    
    
    print accuracies

'print source_sentences[0]\nprint "-----------"\nprint spanish_sentences_universal[0]\nprint "-----------"\ncalculate_accuracy([source_sentences[0]], spanish_sentences_universal, 0)'

In [76]:
def make_alignment(result):
    result_print = []
    for i in range(len(result)):
        result_print.append(str(target[i] + " " + str(result[i])))
    print result_print 
    result_origin = []
    for i in range(len(origin)):
        result_origin.append(str(origin[i].encode('utf-8'))+ " " + str(i))
    print result_origin

In [77]:
pick_test_sentences(100)
save_test_set()
tokenize_test()
tag_test_sentences()
get_source_sentences_tagged()

'pick_test_sentences(100)\nsave_test_set()\ntokenize_test()\ntag_test_sentences()\nget_source_sentences_tagged()'

In [78]:
'''number_of_sentences = [10, 100, 1000, 5000, 10000, 15000, 20000, 25000, 30000]
for i, number in enumerate(number_of_sentences):
    pick_training_sentences(number)
    save_training_set(number)'''

'number_of_sentences = [10, 100, 1000, 5000, 10000, 15000, 20000, 25000, 30000]\nfor i, number in enumerate(number_of_sentences):\n    pick_training_sentences(number)\n    save_training_set(number)'

In [79]:
def load_train_data(number):
    global training_set
    global training_set_2
    global training_set_3
    with open("experiment_data/english_" + str(number) + ".txt", 'rb') as handle:
        training_set = pickle.loads(handle.read())
    with open("experiment_data/german_" + str(number) + ".txt", 'rb') as handle:
        training_set_2 = pickle.loads(handle.read())
    with open("experiment_data/french_" + str(number) + ".txt", 'rb') as handle:
        training_set_3 = pickle.loads(handle.read())

In [80]:
parameters_t_all_IBM1 = None
parameters_t_all_IBM2 = None
parameters_q_all_IBM2 = None

number_of_sentences = [10, 100, 1000, 5000, 10000, 15000, 20000, 25000, 30000]
number_of_trials = 3

def get_data():
    
    global parameters_t_all_IBM1
    global parameters_t_all_IBM2
    global parameters_q_all_IBM2
    
    import time
    start_total_time = time.time()
    
    
    results = []
    for i in range(3):
        results.append([])#IBM1(0), IBM2(1), Fast-align(2)
        for j in range(len(source_sentences[0][0]) + 2):
            results[i].append([])#Directly from Eng(0), Ger(1) and Fre(2), with quorum(3) and with weighted quorum(4)
            results[i][j] = np.zeros(len(number_of_sentences))
    
    
    for i, number in enumerate(number_of_sentences):
        for k in range(number_of_trials):

            if (number != 10000 or k ==2):
                if(k == 0 and i != 0):
                    print "\n\n\n\n\n"
                print "NUMBER OF SENTENCES = ", number

                load_train_data(number)
                tokenize()
                get_types()


                import time
                start_time = time.time()

                ordered_parameters_t_IBM1_eng, parameters_t_IBM1_eng = EM_translation_probabilities(language_train_pairs)
                ordered_parameters_t_IBM1_ger, parameters_t_IBM1_ger = EM_translation_probabilities(language_train_pairs_2)
                ordered_parameters_t_IBM1_fre, parameters_t_IBM1_fre = EM_translation_probabilities(language_train_pairs_3)

                time = time.time() - start_time
                print "\n\nFINISHED ESTIMATING PARAMETERS_IBM1"
                print("--- %s seconds ---" % (time))

                parameters_t_all_IBM1 = []
                parameters_t_all_IBM1.append(parameters_t_IBM1_eng)
                parameters_t_all_IBM1.append(parameters_t_IBM1_ger)
                parameters_t_all_IBM1.append(parameters_t_IBM1_fre)

                dump_model("parameters_t_eng_IBM1_" + str(number) + "_" + str(k) + ".txt", parameters_t_IBM1_eng)
                dump_model("parameters_t_ger_IBM1_" + str(number) + "_" + str(k) + ".txt", parameters_t_IBM1_ger)
                dump_model("parameters_t_fre_IBM1_" + str(number) + "_" + str(k) + ".txt", parameters_t_IBM1_fre)



                '''import time
                start_time = time.time()
                #print "ENGLISH"
                ordered_parameters_t_IBM2_eng, parameters_t_IBM2_eng, parameters_q_IBM2_eng = EM_translation_probabilities_IBM2(language_train_pairs)
                #print "GERMAN"
                ordered_parameters_t_IBM2_ger, parameters_t_IBM2_ger, parameters_q_IBM2_ger = EM_translation_probabilities_IBM2(language_train_pairs_2)
                #print "FRENCH"
                ordered_parameters_t_IBM2_fre, parameters_t_IBM2_fre, parameters_q_IBM2_fre = EM_translation_probabilities_IBM2(language_train_pairs_3)

                time = time.time() - start_time
                print "\nFINISHED ESTIMATING PARAMETERS_IBM2"
                print("--- %s seconds ---" % (time))


                parameters_t_all_IBM2 = []
                parameters_q_all_IBM2 = []

                parameters_t_all_IBM2.append(parameters_t_IBM2_eng)
                parameters_t_all_IBM2.append(parameters_t_IBM2_eng)
                parameters_t_all_IBM2.append(parameters_t_IBM2_eng)

                parameters_q_all_IBM2.append(parameters_q_IBM2_eng)
                parameters_q_all_IBM2.append(parameters_q_IBM2_eng)
                parameters_q_all_IBM2.append(parameters_q_IBM2_eng)


                dump_model("parameters_t_eng_IBM2_" + str(number) + "_" + str(k) + ".txt", parameters_t_IBM2_eng)
                dump_model("parameters_t_ger_IBM2_" + str(number) + "_" + str(k) + ".txt", parameters_t_IBM2_ger)
                dump_model("parameters_t_fre_IBM2_" + str(number) + "_" + str(k) + ".txt", parameters_t_IBM2_fre)     

                dump_model("parameters_q_eng_IBM2_" + str(number) + "_" + str(k) + ".txt", parameters_q_IBM2_eng)
                dump_model("parameters_q_ger_IBM2_" + str(number) + "_" + str(k) + ".txt", parameters_q_IBM2_ger)
                dump_model("parameters_q_fre_IBM2_" + str(number) + "_" + str(k) + ".txt", parameters_q_IBM2_fre)'''

            
            accuracies = calculate_accuracy(source_sentences, spanish_sentences_universal)
            for j in range(len(results[0])):
                results[0][j][i] += accuracies[j]
        
        
    for j in range(len(results[0])):
        results[0][j] /= number_of_trials
        
    print number_of_sentences
    print results[0]
    
    plt.figure(1)
    plt.subplot(211)
    plt.plot(number_of_sentences, results[0][0], 'cs',
             number_of_sentences, results[0][1], 'bs',
             number_of_sentences, results[0][2], 'ks',
             number_of_sentences, results[0][3], 'gs',
             number_of_sentences, results[0][4], 'rs')
    
    import time
    time = time.time() - start_total_time
    print "\nFINISHED GETTING DATA"
    print("--- %s seconds ---" % (time))
        
get_data()

NUMBER OF SENTENCES =  10000

Number of English types =  8423
Number of German types  =  12901
Number of French types  =  15872
Number of Spanish types =  16672


KeyboardInterrupt: 